In [1]:
import sys
sys.path.append('../src')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential, Input, layers
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow_addons.metrics import FBetaScore
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
import tensorflow as tf
# from tensorflow.keras.metrics import Recall, Precision

from utils.data import get_best_slice, select_tomo_ids, get_csv_from_bq
from utils.render_motor_position import get_motor_coordinates, get_slice_file_path
from ml_logic.interface import train_classification


/Users/wm/.pyenv/versions/3.10.6/envs/byu/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/Users/wm/.pyenv/versions/3.10.6/envs/byu/lib/python3.10/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.13.0 and strictly below 2.16.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter 


Loading TensorFlow...


In [2]:
path_train_csv= '../data/csv_raw/train_labels.csv'
# path_image= '../data/pictures_raw/train'
path_image = '../data/pictures_process/adaptequal_1_padded'

df = pd.read_csv(path_train_csv).copy()


In [3]:
# pd.serie with list of tomo (tomo_2dd6bd)
tomogram_id= select_tomo_ids(df,
                             number_of_slices= list(df['Array_shape_axis_0'].unique()) #[500, 800, 600, 300, 400, 494]
                             )


In [4]:
# convert the img in array, ready for input the model

def load_images_by_ids(folder_path,
                       tomo_ids,
                       color_mode='grayscale',
                       target_size=None
                       ):

    images = []
    folder = Path(folder_path)

    for tomo_id in tomo_ids:
        img_file = folder / f"{tomo_id}.jpg"
        if img_file.exists():
            img = load_img(img_file, color_mode=color_mode, target_size=target_size)
            img_array = img_to_array(img) / 255.0
            images.append(img_array)
        else:
            print(f"[⚠️] Imagen no encontrada: {img_file}")

    return np.array(images)


In [5]:
# X and y
X = load_images_by_ids(path_image, tomogram_id)

df_filtered = df[df['tomo_id'].isin(tomogram_id)]
y = df_filtered['Number_of_motors']


In [6]:
# train - Val - test

random_state=42

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.1,
                                                    random_state=random_state)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  test_size=0.1,
                                                  random_state=random_state)

In [7]:

def init_model_densenet(X):
    original_input_shape = X.shape[1:]

    #  force input to 3 channel if it's grey (1 canal)
    if original_input_shape[-1] == 1:
        base_input_shape = original_input_shape[:-1] + (3,)
    else:
        base_input_shape = original_input_shape

    # load DenseNet121 without last classify layer
    base_model = DenseNet121(include_top=False,
                             weights='imagenet',
                             input_shape=base_input_shape
                             )
    base_model.trainable = False

    # Input layer
    inputs = Input(shape=original_input_shape)

    # convert in RGB if need it,
    if original_input_shape[-1] == 1:
        x = Lambda(lambda x: tf.image.grayscale_to_rgb(x))(inputs)
    else:
        x = inputs

    x = base_model(x, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs)

    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=1e-4),
        metrics=[
            'accuracy',
            FBetaScore(num_classes=1,
                       average='micro',
                       beta=2.0),
            Recall()
        ]
    )

    return model


In [8]:
model_densenet = init_model_densenet(X_train)
history_densenet = train_classification(model=model_densenet,
                                        X_train = X_train,
                                        y_train = y_train,
                                        X_val = X_val,
                                        y_val = y_val,
                                        model_type = 'classification',
                                        preprocess_type ='adaptequela_1',
                                        model_name= 'model_densenet121',
                                        batch_size= 16,
                                        patience = 2
                                        )



🏋️ Starting model training ...


2025/06/07 22:51:58 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'


Epoch 1/10


2025-06-07 22:51:59.230301: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


30/30 [==============================] - 398s 13s/step - loss: 0.8059 - accuracy: 0.5385 - fbeta_score: 0.8631 - recall: 0.4789 - val_loss: 0.8432 - val_accuracy: 0.4423 - val_fbeta_score: 0.7986 - val_recall: 1.0000
Epoch 2/10
30/30 [==============================] - 408s 14s/step - loss: 0.6618 - accuracy: 0.6261 - fbeta_score: 0.8631 - recall: 0.8046 - val_loss: 0.6811 - val_accuracy: 0.5962 - val_fbeta_score: 0.7986 - val_recall: 0.9565
Epoch 3/10
30/30 [==============================] - 451s 15s/step - loss: 0.6147 - accuracy: 0.6838 - fbeta_score: 0.8631 - recall: 0.7854 - val_loss: 0.6361 - val_accuracy: 0.6923 - val_fbeta_score: 0.7986 - val_recall: 0.9565
Epoch 4/10
30/30 [==============================] - 427s 14s/step - loss: 0.5574 - accuracy: 0.7201 - fbeta_score: 0.8631 - recall: 0.7854 - val_loss: 0.6297 - val_accuracy: 0.6923 - val_fbeta_score: 0.7986 - val_recall: 0.9565
Epoch 5/10
30/30 [==============================] - 424s 14s/step - loss: 0.5403 - accuracy: 0.7329

...

Cheack point matrics: dict_keys(['loss', 'accuracy', 'fbeta_score', 'recall', 'val_loss', 'val_accuracy', 'val_fbeta_score', 'val_recall'])
✅ Results saved on mlflow


INFO:tensorflow:Assets written to: /var/folders/nr/0rc2wzzd26zd1tptqppt9kjr0000gn/T/tmp2xyb9zbi/model/data/model/assets


INFO:tensorflow:Assets written to: /var/folders/nr/0rc2wzzd26zd1tptqppt9kjr0000gn/T/tmp2xyb9zbi/model/data/model/assets
python(34238) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
2025/06/07 23:56:18 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/nr/0rc2wzzd26zd1tptqppt9kjr0000gn/T/tmp2xyb9zbi/model, flavor: tensorflow). Fall back to return ['tensorflow==2.10.0']. Set logging level to DEBUG to see the full traceback. 
2025/06/07 23:56:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✅ Model saved to mlflow
✅ Model model_densenet121 (version 1) transitioned from None to staging
✅ train() done 

✅ mlflow_run auto-log done


Successfully registered model 'model_densenet121'.
Created version '1' of model 'model_densenet121'.
/Users/wm/code/mc-will/bacterial_motor/BYU_Locating-Bacterial_Flagellar_Motors_2025/notebooks/../src/ml_logic/registry.py:41: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  version = client.get_latest_versions(name=model_name, stages=[current_stage])
/Users/wm/code/mc-will/bacterial_motor/BYU_Locating-Bacterial_Flagellar_Motors_2025/notebooks/../src/ml_logic/registry.py:47: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry sta

In [9]:
model_densenet.evaluate(X_test,y_test)

2/2 [==============================] - 59s 26s/step - loss: 0.5082 - accuracy: 0.7931 - fbeta_score: 0.8333 - recall: 0.9655


[0.5082195401191711,
 0.7931034564971924,
 0.8333333730697632,
 0.9655172228813171]

In [22]:
type(history_densenet)


NoneType

In [25]:
# # Fonction of grafic all the metrics and its val's
# def plot_training_metrics(history):
#     """
#    plot the metrics, even if there is not one of them into 'history'
#     """
#     history_dict = history.history
#     plotted = False

#     def safe_plot(metric_name, color, linestyle='-'):
#         nonlocal plotted
#         if metric_name in history_dict:
#             label = metric_name.replace('_', ' ')
#             plt.plot(history_dict[metric_name], c=color, linestyle=linestyle, label=label)
#             plotted = True

#     # fbeta
#     safe_plot('fbeta_score', 'r')
#     safe_plot('val_fbeta_score', 'r', '--')

#     # recall
#     safe_plot('recall', 'g')
#     safe_plot('val_recall', 'g', '--')
#     safe_plot('recall_1', 'g')  # alternativa por si usa nombre automático
#     safe_plot('val_recall_1', 'g', '--')

#     # accuracy
#     safe_plot('accuracy', 'black')
#     safe_plot('val_accuracy', 'black', '--')

#     # precision
#     safe_plot('precision', 'blue')
#     safe_plot('val_precision', 'blue', '--')

#     if plotted:
#         plt.title("Training Metrics")
#         plt.xlabel("Epoch")
#         plt.ylabel("Metric Value")
#         plt.legend()
#         plt.grid(True)
#         plt.show()
#     else:
#         print("⚠️ No se encontró ninguna métrica conocida para graficar.")


# plot_training_metrics(history_densenet)

In [1]:
from sklearn.metrics import fbeta_score

# La pred dans set de validación
y_val_pred_prob = model_densenet.predict(X_val)
y_val_pred = (y_val_pred_prob > 0.5).astype(int)

# Cal F-beta ponderé
fbeta = fbeta_score(y_val, y_val_pred, beta=2, average='binary')
print(f"F-beta score ponderé(beta=2) en val: {fbeta:.4f}")

NameError: name 'model_densenet' is not defined

In [2]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
models_info = client.search_registered_models()
for model in models_info:
    print(model.name)

model_densenet121
